In [1]:
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials, MomentTools
using QuadGK
using MosekTools
using JuMP
using Random

# Quantum system

In [2]:
#= 
Quantum system is taken from 
 https://github.com/q-optimize/c3/blob/master/examples/two_qubits.ipynb

Drift Hamiltonian
=#

H0 = [
    0 0 0;
    0 3.21505101e+10 0;
    0 0 6.23173079e+10
];

H0 ./= norm(H0, Inf)

# Control Hamiltonian
V = [
    0 1 0;
    1 0 1.41421356;
    0 1.41421356 0
]

V ./= norm(V, Inf);

In [3]:
H0

3×3 Matrix{Float64}:
 0.0  0.0       0.0
 0.0  0.515916  0.0
 0.0  0.0       1.0

# Symbolic integration utilities (had to implement them by myself)

In [4]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        coeff * term * x * 1 // (exponents(term)[ind_x] + 1)
        for (coeff, term) in zip(coefficients(p), terms(p));
        init = 0 * x
    )
        
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))
    )
end

function square_fro_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

square_fro_norm (generic function with 1 method)

In [217]:
@polyvar x[1:3]
@polyvar t[1:3]

#Random.seed!(62220221)

# final time
#T = 1.

@polyvar T

function u(t, x)
    # the polynomial shape for control
    sum(x[n] * t^(n - 1) for n = 1:length(x))
end

function get_unitray(x::AbstractArray)
    T = 0.1
    
    b = NLevelBasis(size(H0)[1])

    𝓗₀ = DenseOperator(b, b, H0)
    𝓥 = DenseOperator(b, b, V)

    H = LazySum([1., u(0, x)], [𝓗₀, 𝓥])
    
    function 𝓗(t, psi)
        H.factors[2] = u(t, x)
        return H
    end

    _, 𝓤 = timeevolution.schroedinger_dynamic([0:T;], identityoperator(b,b), 𝓗)

    return Matrix(𝓤[2].data)
    #=
    T = 0.1
    
    dt = 0.001
    
    U = I + 0im * H0 
    
    for t in dt/2:dt:T
       U = exp(-im * dt * (H0 + u(t, x) * V)) * U
    end
    
    return U
    =#
end
###############################################################################
# Get the target unitary


# randomly generate the coefficients of the polynomial control
#exact_x = randn(length(x)) # normal random variable


# target unitray
U_target = get_unitray(exact_x)


###############################################################################
# Get the polynomial objective function

function A(t, x)
    #=
    The generator of motion entering the Magnus expansion
    =#
    (H0 + V * u(t, x)) / im
end

function commutator(a, b)
    a * b - b * a
end 

# Convergence test for the Magnus expansion (convergence_test < 1)
#convergence_test = quadgk(t -> opnorm(A(t, exact_x)), 0, T)[1] / π

###############################################################################
# get the partial sum of the Magnus expansion
A₁ = A(t[1], x)
A₂ = A(t[2], x)

Ω = ∫(A₁, t[1], 0, T)

# 2nd term in the Magnus expansion
Ω .+= 1//2 * ∫(∫(
    commutator(A₁, A₂), 
    t[2], 0, t[1]), 
    t[1], 0, T
)

# 3nd term in the Magnus expansion

A₃ = A(t[3], x)

Ω .+= 1//6 * ∫(∫(∫(
    commutator(A₁, commutator(A₂, A₃)) + commutator(commutator(A₁, A₂), A₃),
    t[3], 0, t[2]),
    t[2], 0, t[1]),
    t[1], 0, T
)

###############################################################################


# In the next section we evaluate the above approximation for \tanh(\Omega/2) and save it as approx_tanh.
# Note that we empirically found that it is good to keep first 3 terms in the Taylor expansion for tanh

Ω = convert(typeof(A₁), Ω)

Ω²¼ = Ω^2 / 4

approx_tanh = (Ω/2) * (I + Ω²¼ * (-1//3 * I + 2//15 * Ω²¼))


obj = square_fro_norm(
    I - U_target + (I + U_target) * approx_tanh
    #log(U_target) - Ω
)

# save the value of objective function for exact x
#obj_exact_x = obj(exact_x)

###############################################################################
# Perform polynomial optimization

#@assert variables(obj) == x

optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
obj_min, M = minimize(obj, [], [0.5 - T, T], variables(obj), maxdegree(obj) ÷ 2, optimizer)

LoadError: BoundsError: attempt to access 0-element Vector{Operator{NLevelBasis{Int64}, NLevelBasis{Int64}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}} at index [2]

In [180]:
variables(obj)

4-element Vector{PolyVar{true}}:
 x₁
 x₂
 x₃
 T

In [179]:
obj_exact_x

2.541167602152938

In [146]:
convergence_test

0.4507924035474972

In [132]:
get_minimizers(M)

3×241 Matrix{Float64}:
 -3.10061  -2.98947   -2.90161   -2.9979   …   2.90162   2.98955   3.10063
 -1.11644  -0.495674   0.142567  -0.68291     -0.142763  0.495452  1.11603
 -2.52253  -2.49078   -2.38833   -1.86963      2.38847   2.49071   2.52265

In [185]:
w, Xi = get_measure(M)

([2.7214559180044705e-19, 8.581778971021911e-16, 9.632676602618055e-17, 7.035921769226612e-16, 1.7107957167209297e-15, 1.7219494291563613e-16, 4.113575057597948e-16, 1.5358705673174346e-16, 3.550607047833607e-16, 4.4006751712742396e-15  …  1.0178472977057233e-10, 8.790001100999401e-10, 4.2049391663170685e-12, 2.7158625810006386e-11, 1.7995046607633784e-12, 4.451150946023366e-14, 1.6658213997672362e-15, 1.677305808634037e-14, 3.911540735772152e-15, 6.856665701570541e-16], [-9.993786845698502 -7.913425934259114 … 7.871190088879646 10.149164031263325; -12.501859124045604 -9.084950844838366 … 8.306326714363362 10.815967595542352; -5.7438466222492846 -2.645870980890246 … 2.5167052632264224 6.778305393876472; -13.29534401975623 -12.627361992620475 … -8.685380828786144 -10.114056518504531])

In [140]:
size(Xi)

(3, 241)

In [199]:
r = real.(get_minimizers(M))
obj_min_vals = [obj(r[:,i]) for i=1:size(r)[2]]
min_x = r[:, argmin(obj_min_vals)]

4-element Vector{Float64}:
 -0.2034801851699316
 -0.07035917899766514
  0.27199699062495286
  0.01958551115629629

In [200]:
min(obj_min_vals...)

0.01334516485838468

In [186]:
obj_min_vals = [obj(Xi[:,i]) for i=1:size(Xi)[2]]
min_x_ = Xi[:, argmin(obj_min_vals)]

4-element Vector{Float64}:
  0.46246644047819585
 -0.07388569531009245
 -0.15287708389055032
 -0.5167645013357564

In [143]:
min(obj_min_vals...)

0.023551129684984694

In [133]:
r = real.(get_minimizers(M))
obj_min_vals = [obj(r[:,i]) for i=1:size(r)[2]]
min_x = r[:, argmin(obj_min_vals)]

3-element Vector{Float64}:
 -0.00021112063660367003
 -2.192017298207576e-5
 -0.00010829015951859395

In [109]:
exact_x

3-element Vector{Float64}:
 -0.0717071233852124
 -0.2605954047030883
 -1.0028083436087574

In [110]:
min(obj_min_vals...)

0.023551129641367963

In [191]:
using NLopt

vars = variables(obj)

function g(a...)
    # Converting polynomial expression to function to be minimize
    obj(vars => a)
end
    
model = Model(NLopt.Optimizer)

set_optimizer_attribute(model, "algorithm", :LD_MMA)

#set_silent(model)
@variable(model, y[1:length(vars)])

# random guess
for (var, init_val) in zip(y, min_x)
    set_start_value(var, init_val)
end

register(model, :g, length(y), g; autodiff = true)
@NLobjective(model, Min, g(y...))
JuMP.optimize!(model)

solution = vars => map(value, y)

objective_value(model)

0.012894435328329712

In [192]:
solution

PolyVar{true}[x₁, x₂, x₃, T] => [0.03981957803342788, -0.008760494558734249, -0.00433830953674443, -2.4332881628463334e-7]

In [193]:
exact_x

3-element Vector{Float64}:
 -0.0717071233852124
 -0.2605954047030883
 -1.0028083436087574

In [218]:
norm(get_unitray(map(value, y)[1:end - 1]) - U_target)

LoadError: BoundsError: attempt to access 0-element Vector{Operator{NLevelBasis{Int64}, NLevelBasis{Int64}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}} at index [2]

In [213]:
norm(get_unitray(randn(3)) - U_target)

0.29477468047857935

In [202]:
4 * 300

1200

In [1]:
Threads.nthreads()

1